In [63]:
import pandas as pd 
from dataclasses import dataclass
from typing import List
from datetime import datetime


In [64]:
df_nov=pd.read_excel(r"..\Data\LineC_SessionData-individual rats_SampleData.xlsx")
df_march=pd.read_excel(r"..\Data\Snifftime data_March15.2019_example_withIndividualSniffs.xlsx")
df_sept=pd.read_excel(r"..\Data\AutomatedCage_Sept2023-24_raw data.xlsx")

In [65]:
df_sept.columns

Index(['RAT_NAME', 'ID_SAMPLE', 'LEVEL_NAME', 'RUN', 'HOLE', 'ID_BL_DOTS',
       'ID_GXP_DOTS', 'ID_BL_APOPO', 'EXECUTED', 'HIT', 'REWARD', 'SniffTime',
       'ReadTotalSnifftime', 'SESSION_DATE', 'ID_EVALUATION_SESSION',
       'STATUS_BLINDPOS', 'SAMPLE_TYPE', 'tblRAT_SESSION_REMARKS', 'REUSED',
       'tblEVALUATION_SniffThreshold', 'tblRAT_SESSION_SniffThreshold',
       'CONFIGURATION_NAME', 'TEMPERATURE', 'STATUS_KNOWNPOS', 'START_TIME',
       'END_TIME', 'DATE_INCOMING', 'tblEVALUATION_SESSION_REMARKS', 'Trainer',
       'Documenter', 'Handler', 'DOTS_NAME', 'ID_CONFIGURATION'],
      dtype='object')

In [66]:
rename_dict = {
    "RAT_NAME": "Rat_Name",
    "ID_EVALUATION_SESSION": "Evaluation_Session_ID",
    "RUN": "Run",
    "HOLE": "Hole",
    "SniffTime": "Sniff_Time",
    "tblEVALUATION_SniffThreshold": "Threshold_Time",
    "SESSION_DATE" : "Date"
    # "ID_BL_DOTS": "Lab_Test"
}

In [67]:
df_sept = df_sept.rename(columns=rename_dict)

In [68]:
def _positive_sample_rat(row): 
    if row['Sniff_Time']>=row['Threshold_Time']: 
        return 1 
    else: 
        return 0

def _positive_sample_lab(row): 
    if row['ID_BL_DOTS']==0:
        return pd.NA
    elif row['ID_BL_DOTS']>1: 
        return 1
    else: 
        return 0


def false_positive(rat, lab):
    """Return 1 if rat says positive but lab test is negative, else 0"""
    return 1 if rat == 1 and lab == 0 else 0

def false_negative(rat, lab):
    """Return 1 if rat says negative but lab test is positive, else 0"""
    return 1 if rat == 0 and lab == 1 else 0

def true_positive(rat, lab):
    """Return 1 if rat says positive and lab test is positive, else 0"""
    return 1 if rat == 1 and lab == 1 else 0

def true_negative(rat, lab):
    """Return 1 if rat says negative and lab test is negative, else 0"""
    return 1 if rat == 0 and lab == 0 else 0


In [70]:
df_sept['Result_Of_Rat']=df_sept.apply(lambda x: _positive_sample_rat(x),axis=1)
df_sept['Lab_Test']=df_sept.apply(lambda x: _positive_sample_lab(x),axis=1)
df_sept["FP"] = df_sept.apply(lambda row: false_positive(row["Result_Of_Rat"], row["Lab_Test"]), axis=1)
df_sept["FN"] = df_sept.apply(lambda row: false_negative(row["Result_Of_Rat"], row["Lab_Test"]), axis=1)
df_sept["TP"]  = df_sept.apply(lambda row: true_positive(row["Result_Of_Rat"], row["Lab_Test"]), axis=1)
df_sept["TN"]  = df_sept.apply(lambda row: true_negative(row["Result_Of_Rat"], row["Lab_Test"]), axis=1)
df_sept["Age"]=""
df_sept["Weight"]=""
df_sept["Gender"]=""

In [88]:
sum(df_sept['FP'].isna())

0

In [71]:
# Select and rename columns, add empty columns, and calculate new columns for df_march
df_march_mod = df_march.rename(columns={
    'RAT_NAME': 'Rat_Name',
    'ID_EVALUATION_SESSION': 'Evaluation_Session_ID',
    'age': 'Age',
    'RUN': 'Run',
    'HOLE': 'Hole',
    'Sniff 1': 'Sniff_Time',
    'tblRAT_SESSION.SniffThreshold': 'Threshold_Time',
    "SESSION_DATE" : "Date"
})

# Keep only the required columns (including those to be created)
columns_needed = [
    'Rat_Name',
    'Evaluation_Session_ID',
    'Gender',
    'Age',
    'Weight',
    'Run',
    'Hole',
    'Sniff_Time',
    'Threshold_Time',
    'Result_Of_Rat',
    'Lab_Test',
    "Date"
]

# Add empty columns for Gender and Weight
df_march_mod['Gender'] = ''
df_march_mod['Weight'] = ''
df_march_mod['Sniff_Time'] = df_march_mod['Sniff_Time'].fillna(0)

# Calculate Result_Of_Rat: 1 if Sniff_Time > Threshold_Time, else 0
df_march_mod['Result_Of_Rat'] = (df_march_mod['Sniff_Time'] > df_march_mod['Threshold_Time']).astype(int)

# Calculate Lab_Test: 1 if ID_BL_DOTS > 1, else 0 if == 1
df_march_mod['Lab_Test'] = df_march['ID_BL_DOTS'].apply(lambda x: 1 if x > 1 else (0 if x == 1 else None))

# Reorder columns to match the requested order
df_march_mod = df_march_mod[columns_needed]

# Display the first few rows to check
df_march_mod.head()

,Rat_Name,Evaluation_Session_ID,Gender,Age,Weight,Run,Hole,Sniff_Time,Threshold_Time,Result_Of_Rat,Lab_Test,Date
0,Daudi,14097,,2.38,,A,1,0.0,2500,0,0,2019-03-15
1,Daudi,14097,,2.38,,A,2,3831.0,2500,1,1,2019-03-15
2,Daudi,14097,,2.38,,A,3,0.0,2500,0,0,2019-03-15
3,Daudi,14097,,2.38,,A,4,0.0,2500,0,0,2019-03-15
4,Daudi,14097,,2.38,,A,5,0.0,2500,0,0,2019-03-15


In [72]:
# Create FP, FN, TP, TN columns based on Lab_Test (ground truth) and Result_Of_Rat (prediction)
df_march_mod['TP'] = ((df_march_mod['Lab_Test'] == 1) & (df_march_mod['Result_Of_Rat'] == 1)).astype(int)
df_march_mod['TN'] = ((df_march_mod['Lab_Test'] == 0) & (df_march_mod['Result_Of_Rat'] == 0)).astype(int)
df_march_mod['FP'] = ((df_march_mod['Lab_Test'] == 0) & (df_march_mod['Result_Of_Rat'] == 1)).astype(int)
df_march_mod['FN'] = ((df_march_mod['Lab_Test'] == 1) & (df_march_mod['Result_Of_Rat'] == 0)).astype(int)

df_march_mod.head()

,Rat_Name,Evaluation_Session_ID,Gender,Age,Weight,Run,Hole,Sniff_Time,Threshold_Time,Result_Of_Rat,Lab_Test,Date,TP,TN,FP,FN
0,Daudi,14097,,2.38,,A,1,0.0,2500,0,0,2019-03-15,0,1,0,0
1,Daudi,14097,,2.38,,A,2,3831.0,2500,1,1,2019-03-15,1,0,0,0
2,Daudi,14097,,2.38,,A,3,0.0,2500,0,0,2019-03-15,0,1,0,0
3,Daudi,14097,,2.38,,A,4,0.0,2500,0,0,2019-03-15,0,1,0,0
4,Daudi,14097,,2.38,,A,5,0.0,2500,0,0,2019-03-15,0,1,0,0


In [78]:
# Select and rename only the columns you want
df_nov_selected = df_nov.rename(columns={
    'Name': 'Rat_Name',
    'ID_EVALUATION_SESSION': 'Evaluation_Session_ID',
    'Gender': 'Gender',
    'WEIGHT': 'Weight',
    'SniffTime': 'Sniff_Time',
    'SniffThreshold': 'Threshold_Time',
    'ID_BL_DOTS': 'ID_BL_DOTS',
    "SESSION_DATE" : "Date"
}).copy()

# Calculate Age
df_nov_selected['Age'] = ((
    pd.to_datetime(df_nov['SESSION_DATE']) - pd.to_datetime(df_nov['Birthdate'])
).dt.days / 365.25).round(2)

df_nov_selected['Run'] = ''
df_nov_selected['Hole'] = ''
# Reorder to match your desired structure
final_columns = [
    'Rat_Name', 'Evaluation_Session_ID', 'Gender', 'Age', 'Weight',
    'Run', 'Hole', 'Sniff_Time', 'Threshold_Time', 'ID_BL_DOTS',"Date"
]

df_nov_final = df_nov_selected[final_columns]

df_nov_final = df_nov_final.copy()

df_nov_final.loc[:, 'Lab_Test'] = df_nov_final['ID_BL_DOTS'].apply(lambda x: 1 if x == 1 else 0)
df_nov_final = df_nov_final.drop(columns=['ID_BL_DOTS'])

df_nov_final = df_nov_final.copy()
df_nov_final.loc[:, 'Result_Of_Rat'] = (
    df_nov_final['Sniff_Time'] >= df_nov_final['Threshold_Time']
).astype(int)

df_nov_final = df_nov_final.copy()

# FP: Rat says Positive (1), Lab says Negative (0)
df_nov_final['FP'] = ((df_nov_final['Result_Of_Rat'] == 1) & (df_nov_final['Lab_Test'] == 0)).astype(int)

# FN: Rat says Negative (0), Lab says Positive (1)
df_nov_final['FN'] = ((df_nov_final['Result_Of_Rat'] == 0) & (df_nov_final['Lab_Test'] == 1)).astype(int)

# TP: Rat says Positive (1), Lab says Positive (1)
df_nov_final['TP'] = ((df_nov_final['Result_Of_Rat'] == 1) & (df_nov_final['Lab_Test'] == 1)).astype(int)

# TN: Rat says Negative (0), Lab says Negative (0)
df_nov_final['TN'] = ((df_nov_final['Result_Of_Rat'] == 0) & (df_nov_final['Lab_Test'] == 0)).astype(int)

In [93]:
columns_needed = [
    'Rat_Name',
    'Evaluation_Session_ID',
    'Gender',
    'Age',
    'Weight',
    'Run',
    'Hole',
    'Sniff_Time',
    'Threshold_Time',
    'Result_Of_Rat',
    'Lab_Test',
    "Date", 
    "TP",
    "FP",
    "TN",
    "FN"
]

In [94]:
final_df= pd.concat([pd.concat([df_sept[columns_needed],df_march_mod[columns_needed]]),df_nov_final])


In [95]:
final_df

,Rat_Name,Evaluation_Session_ID,Gender,Age,Weight,Run,Hole,Sniff_Time,Threshold_Time,Result_Of_Rat,Lab_Test,Date,TP,FP,TN,FN
0,Ella,22012,,,,A,1,784.0,1000,0,1,2023-09-01,0,0,0,1
1,Ella,22012,,,,A,2,724.0,1000,0,1,2023-09-01,0,0,0,1
2,Ella,22012,,,,A,3,2599.0,0,1,1,2023-09-01,1,0,0,0
3,Ella,22012,,,,B,1,2576.0,1000,1,1,2023-09-01,1,0,0,0
4,Ella,22012,,,,B,2,755.0,1000,0,1,2023-09-01,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34848,Maliwaza,6976,F,3.68,830.0,,,207.0,1600,0,1,2016-12-06,0,0,0,1
34849,Maliwaza,6976,F,3.68,830.0,,,1602.0,1600,1,0,2016-12-06,0,1,0,0
34850,Maliwaza,6976,F,3.68,830.0,,,0.0,1600,0,1,2016-12-06,0,0,0,1
34851,Maliwaza,6976,F,3.68,830.0,,,0.0,1600,0,1,2016-12-06,0,0,0,1


In [98]:
final_df.to_excel(r"..\Data\Report_data.xlsx")